In this tutorial we are doing multi-class classification of mnist-fashion data using simple Convolutional Neural Networks.
We used Keras Sequential API to build our model. There are a total of 70K samples and number of classes is 10. Each image is of size 28*28.  Most of the code is taken from Coursera deep learning course. 

# Import libraries

In [1]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt

1.13.1


# Load dataset

Fashion MNIST data is available in the tf.keras datasets API. load_data function call will give us data in the form of training and testing images along with their labels. Downloading of this dataset took around 10 seconds on my laptop. 

In [2]:
mnist = tf.keras.datasets.fashion_mnist

In [3]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
training_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

###### We can see that there are  a total of 60,000 images for training and 10,000 for testing. Each image size is 28*28

Now for CNN, we need to reshape our training and testing data. That's because the first convolution expects a single tensor containing everything, so instead of 60,000 28x28x1 items in a list, we have a single 4D list that is 60,000x28x28x1, and the same for the test images.

In [5]:
training_images=training_images.reshape(60000, 28, 28, 1)
test_images = test_images.reshape(10000, 28, 28, 1)
training_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

# Data normalization


Normalization is an important step if we are to use Neural networks for classification. Since maximum pixel value is 255, so we will divide all values with 255 to get a value between 0 and 1. 

In [6]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

# Model building using Keras Sequential API

### Callbacks

There might comes a situation where we reach our required loss at some epoc and we don't want to proceed further. Callbacks come handy in such situations. One use of callbacks along with many others is that we can put a threshold on accuracy or loss and model will automatically stop training once it reaches the threshold value. In class below, we are using a thresold value of 0.4 for loss. Model will stop training after minimizing loss to 0.4.


In [7]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.1):
      print("\nMinimized loss to threshold value so cancelling training!")
      self.model.stop_training = True

### Creating a model and adding layers

Next is to define your model. Now instead of the input layer at the top, we are going to add a Convolution. The parameters are:

- The number of convolution filers we want to generate. Purely arbitrary, but good to start with something in the order of 32. Here we are using 32 filters.

- Each filter is of size 3*3.
- We are using ReLU and Softmax activation functions.
- First layer will have input size while last layer will have output size. 


We will follow the Convolution with a MaxPooling layer which is then designed to compress the image, while maintaining the content of the features that were highlighted by the convlution. By specifying (2,2) for the MaxPooling, the effect is to quarter the size of the image. There are different types of pooling filters like max, min, average etc., we are using max pooling in our code. Without going into too much detail here, the idea is that it creates a 2x2 array of pixels, and picks the biggest one, thus turning 4 pixels into 1. It repeats this across the image, and in so doing halves the number of horizontal, and halves the number of vertical pixels, effectively reducing the image by 25%.



#### ReLU:
If X>0, ReLU will return X, else will return 0. 

#### Softmax
It takes a set of values, and effectively picks the biggest one, so, for example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0]. The goal is to save a lot of coding!

#### Output Layer:
tf.keras.layers.Dense(10, activation=tf.nn.softmax)]) is the output layer. Here 10 is the number of output or total number of classes. Here we have 10 classes or target values, so we used 10 in last layer. 


In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

Instructions for updating:
Colocations handled automatically by placer.


### Model Compile


In [10]:

model.compile(optimizer =optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])




In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total para

### Fit Model

In [11]:
callbacks = myCallback()
hist=model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])
#model.fit(training_images, training_labels, epochs=5)


Epoch 1/5
60000/60000 [==============================] - 99s 2ms/sample - loss: 0.4462 - acc: 0.8366
Epoch 2/5
60000/60000 [==============================] - 107s 2ms/sample - loss: 0.2963 - acc: 0.8906s - los
Epoch 3/5
60000/60000 [==============================] - 112s 2ms/sample - loss: 0.2491 - acc: 0.9080
Epoch 4/5
60000/60000 [==============================] - 91s 2ms/sample - loss: 0.2186 - acc: 0.9187
Epoch 5/5
60000/60000 [==============================] - 90s 1ms/sample - loss: 0.1910 - acc: 0.9284


We can see that we got accuracy of 0.9291 or around 93% on training data. We can also check accuracy and loss on test data as well. If accuracy on training data is much higher than of testing data, model is overfitted.

### Evaluating model

We evaluated our model on testing data. We also give single image to our model and verified that our model is giving same class of image as of actual class. 

In [12]:
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
pred_probs=classifications[0]
pred_class=np.where(classifications[0] == np.amax(classifications[0]))

print('Predicted class of test image at index 0 is:',pred_class[0][0])
print('Actual class of test image at index 0 is:',test_labels[0])

10000/10000 [==============================] - 4s 358us/sample - loss: 0.2624 - acc: 0.9044
Predicted class of test image at index 0 is: 9
Actual class of test image at index 0 is: 9


It can be seen that accuaracy on test data is around 90.5%. Since there is not a significant difference in training and testing accuracy, so model is neither overfitted, not underfitted. 
